### 对比区别，年份，orderflow/ level2summary，出报表

### 五分钟线

In [ ]:
import os
import pandas as pd
import numpy as np
import csv

class Folder_Handler(): # 进母文件夹，出子文件夹路径
    
    def __init__(self, mother_path):
        self.mother_path = mother_path
    
    def son_folders(self):
        son_path = []
        sonlist = os.listdir(self.mother_path)
        for son_i in sonlist:
            son_path.append(self.mother_path + '/' + son_i)
        return son_path

class Folder_Handler2(): # 进两个母文件夹，出两个筛选后的子文件夹路径
    
    def __init__(self, mother_path1, mother_path2):
        self.mother_path1 = mother_path1
        self.mother_path2 = mother_path2
    
    def son_folders(self):
        son_path1 = []
        son_path2 = []
        sonlist2 = os.listdir(self.mother_path2)
        for son_i in sonlist2:
            if os.path.exists(self.mother_path2 + '/' + son_i) == True:
                son_path1.append(self.mother_path1 + '/' + son_i)
                son_path2.append(self.mother_path2 + '/' + son_i)
        return son_path1, son_path2
    
class Compare(): # 进两组时间路径，出写入比较后的csv
    
    def __init__(self, time_path1, time_path2, date_i):
        self.time_path1 = time_path1
        self.time_path2 = time_path2
        self.date = date_i
        
    def compare(self):
        df1 = pd.read_csv(self.getpath_f1, sep=",")
        df2 = pd.read_csv(self.getpath_f2, sep=",")
        df1["std"] = 0.0
        df1.set_index('SecuCode', inplace=True)
        df2.set_index('SecuCode', inplace=True)
        df = ((df2 - df1) / df2) * 100
        return df
    
    def gen_df(self):
        for n in range(len(self.time_path2)):
            self.getpath_f1 = self.time_path1[n]
            self.getpath_f2 = self.time_path2[n]
            print(self.getpath_f1)
            print(self.getpath_f2)
            d = self.getpath_f1
            time = d[ d.rindex( '/' ) + 1 : len( d ) ]
            df = self.compare()
            if df.empty == False:
                os.system("pwd") #确认是从yuqin work出发
                os.system("mkdir -p 2020Compare")
                os.chdir("2020Compare")
                os.system("mkdir -p 0119")
                os.chdir("0119")
                os.system("mkdir -p %s" %self.date)
                os.chdir("%s" %self.date)
                os.system("pwd")
                log = open("/home/yuqin/work/2020Compare/0119/%s/%s.csv" %(self.date,time), "w")
                print('log_open')
                df.to_csv(log, sep = ',')
                log.close() 
                print('log_done')
                os.chdir("../../..")
    
class Reports(): # 进一天的全部时间比较文件路径，出一天一个百分比报表
    
    def __init__(self, time_path, date_i):
        self.time_path = time_path
        self.date = date_i
        
    def gen_reports(self):
#         # level2summary
#         headers = ['time', 'AskPriceAtL10Twa', 'AskPriceAtL1Vwa',
#                'AskPriceAtL3Twa', 'AskPriceAtL5Twa', 'AskPriceAvgL10Twa', 'AskPriceAvgL10Vwa', 'AskPriceAvgL3Twa', 'AskPriceAvgL3Vwa',
#                'AskPriceAvgL5Twa', 'AskPriceAvgL5Vwa', 'AskPriceVwaL10Twa', 'AskPriceVwaL3Twa', 'AskPriceVwaL5Twa', 'AskVolSumL10Twa',
#                'AskVolSumL3Twa', 'AskVolSumL5Twa',
#               'BidPriceAtL10Twa', 'BidPriceAtL1Vwa', 'BidPriceAtL3Twa', 'BidPriceAtL5Twa', 'BidPriceAvgL10Twa', 'BidPriceAvgL10Vwa',
#               'BidPriceAvgL3Twa', 'BidPriceAvgL3Vwa', 'BidPriceAvgL5Twa', 'BidPriceAvgL5Vwa', 'BidPriceVwaL10Twa', 'BidPriceVwaL3Twa',
#               'BidPriceVwaL5Twa', 'BidVolSumL10Twa', 'BidVolSumL3Twa', 'BidVolSumL5Twa', 'LastPriceStd', 'PreClosePrice']
        # orderflow
        headers = ['time', 'BuyAmount', 'BuyExlargeOrders', 'BuyExlargeVolume', 'BuyLargeOrders', 'BuyLargeVolume', 'BuyMedOrders', 
           'BuyMedVolume', 'BuyOrders', 'BuySmallOrders', 'BuySmallVolume', 'BuyVolume', 'SellAmount', 'SellExlargeOrders', 
          'SellExlargeVolume', 'SellLargeOrders', 'SellLargeVolume', 'SellMedOrders', 'SellMedVolume', 'SellOrders', 
          'SellSmallOrders', 'SellSmallVolume', 'SellVolume']
        
        OneDay_logname = "/home/yuqin/work/2020Compare/0119/reports/" + str(self.date) + ".csv"
        OneDay_log = open(OneDay_logname , 'w')
        log = csv.DictWriter(OneDay_log, headers)
        log.writeheader()
        print('log opened')

        for n in range(len(self.time_path)):
            df = pd.read_csv(self.time_path[n], sep=",")
            df_colname = list(df.columns)
#             col = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]  # level2summary
            col = range(1,len(df_colname)-1) # orderflow
            puredata = df.values 
            print(self.time_path[n])
            d = self.time_path[n]
            time = d[ d.rindex( '/' ) + 1 : len( d ) ]
            time = str(time[0:5])
            ratio = []
            for j in range(len(col)):
                ratio.append(0)
                total = 0
                for i in range(puredata.shape[0]):
                    total += 1
                    if np.isnan(puredata[i][col[j]]) == True:
                        puredata[i][col[j]] = 0  
                    if abs(puredata[i][col[j]]) >= 0.1:
                        ratio[j] += 1
                ratio[j] = (ratio[j]/total) *100
#             # level2summary
#             log.writerow({'time':str(time), 'AskPriceAtL10Twa':str(ratio[0])+'%', 'AskPriceAtL1Vwa':str(ratio[1])+'%', 
#                       'AskPriceAtL3Twa':str(ratio[2])+'%', 'AskPriceAtL5Twa':str(ratio[3])+'%', 'AskPriceAvgL10Twa':str(ratio[4])+'%', 
#                       'AskPriceAvgL10Vwa':str(ratio[5])+'%', 'AskPriceAvgL3Twa':str(ratio[6])+'%','AskPriceAvgL3Vwa':str(ratio[7])+'%', 
#                       'AskPriceAvgL5Twa':str(ratio[8])+'%', 'AskPriceAvgL5Vwa':str(ratio[9])+'%', 'AskPriceVwaL10Twa':str(ratio[10])+'%', 
#                       'AskPriceVwaL3Twa':str(ratio[11])+'%', 'AskPriceVwaL5Twa':str(ratio[12])+'%', 'AskVolSumL10Twa':str(ratio[13])+'%', 
#                       'AskVolSumL3Twa':str(ratio[14])+'%', 'AskVolSumL5Twa':str(ratio[15])+'%', 'BidPriceAtL10Twa':str(ratio[16])+'%', 
#                       'BidPriceAtL1Vwa':str(ratio[17])+'%', 'BidPriceAtL3Twa':str(ratio[18])+'%', 'BidPriceAtL5Twa':str(ratio[19])+'%', 
#                       'BidPriceAvgL10Twa':str(ratio[20])+'%', 'BidPriceAvgL10Vwa':str(ratio[21])+'%', 'BidPriceAvgL3Twa':str(ratio[22])+'%', 
#                       'BidPriceAvgL3Vwa':str(ratio[23])+'%', 'BidPriceAvgL5Twa':str(ratio[24])+'%', 'BidPriceAvgL5Vwa':str(ratio[25])+'%', 
#                       'BidPriceVwaL10Twa':str(ratio[26])+'%', 'BidPriceVwaL3Twa':str(ratio[27])+'%', 'BidPriceVwaL5Twa':str(ratio[28])+'%', 
#                       'BidVolSumL10Twa':str(ratio[29])+'%', 'BidVolSumL3Twa':str(ratio[30])+'%','BidVolSumL5Twa':str(ratio[31])+'%', 
#                       'LastPriceStd':str(ratio[32])+'%', 'PreClosePrice':str(ratio[33])+'%'})
            # orderflow
            log.writerow({'time':str(time), 'BuyAmount':str(ratio[0])+'%', 'BuyExlargeOrders':str(ratio[1])+'%',  'BuyExlargeVolume':str(ratio[2])+'%', 
                  'BuyLargeOrders':str(ratio[3])+'%', 'BuyLargeVolume':str(ratio[4])+'%', 'BuyMedOrders':str(ratio[5])+'%', 'BuyMedVolume':str(ratio[6])+'%', 
                  'BuyOrders':str(ratio[7])+'%', 'BuySmallOrders':str(ratio[8])+'%', 'BuySmallVolume':str(ratio[9])+'%', 'BuyVolume':str(ratio[10])+'%', 
                  'SellAmount':str(ratio[11])+'%', 'SellExlargeOrders':str(ratio[12])+'%', 'SellExlargeVolume':str(ratio[13])+'%', 'SellLargeOrders':str(ratio[14])+'%', 
                 'SellLargeVolume':str(ratio[15])+'%', 'SellMedOrders':str(ratio[16])+'%', 'SellMedVolume':str(ratio[17])+'%', 'SellOrders':str(ratio[18])+'%', 
                 'SellSmallOrders':str(ratio[19])+'%', 'SellSmallVolume':str(ratio[20])+'%', 'SellVolume':str(ratio[21])+'%'})
        print('log done')


def main():
    
#     for year in range(2012, 2021):
#         datelist = os.listdir("/dat/all_ref/Equity/CN/marketdata/level2summary/5min/data/" + str(year))
#     #往下全部缩进一格, main中以天为单位循环
#     datelist = [20200803,20200806,20200826]
    datelist = os.listdir("/dat/all_ref/Equity/CN/marketdata/level2summary/5min/data/2020" )
    for date_i in datelist:
        # Compare()
        f1 = "/dat/all_till20201230/Equity/CN/marketdata/orderflow/5min/data/2020/" + str(date_i)
        f2 = "/dat/all_ref/Equity/CN/marketdata/orderflow/5min/data/2020/" + str(date_i)
        path = Folder_Handler2(f1, f2)
        time_path1, time_path2= path.son_folders()
        compare_res = Compare(time_path1, time_path2, date_i)
        compare_res.gen_df()

        # Reports()
        f = "/home/yuqin/work/2020Compare/0119/" + str(date_i)
        path = Folder_Handler(f)
        time_path = path.son_folders()
        reports = Reports(time_path, date_i)
        reports.gen_reports()
                
                
if __name__ == '__main__':
    
    main()

### 日线

In [ ]:
import os
import pandas as pd
import numpy as np
import csv

class Folder_Handler(): # 进母文件夹，出子文件夹路径
    
    def __init__(self, mother_path):
        self.mother_path = mother_path
    
    def son_folders(self):
        son_path = []
        sonlist = os.listdir(self.mother_path)
        for son_i in sonlist:
            son_path.append(self.mother_path + '/' + son_i)
        return son_path

class Folder_Handler2(): # 进两个母文件夹，出两个筛选后的子文件夹路径
    
    def __init__(self, mother_path1, mother_path2):
        self.mother_path1 = mother_path1
        self.mother_path2 = mother_path2
    
    def son_folders(self):
        son_path1 = []
        son_path2 = []
        sonlist2 = os.listdir(self.mother_path2)
        for son_i in sonlist2:
            if os.path.exists(self.mother_path2 + '/' + son_i) == True:
                son_path1.append(self.mother_path1 + '/' + son_i)
                son_path2.append(self.mother_path2 + '/' + son_i)
        return son_path1, son_path2
    
class Compare(): # 进两组时间路径，出写入比较后的csv
    
    def __init__(self, f1, f2, date_i):
        self.getpath_f1 = f1
        self.getpath_f2 = f2
        self.date = date_i
        
    def compare(self):
        df1 = pd.read_csv(self.getpath_f1, sep=",")
        df2 = pd.read_csv(self.getpath_f2, sep=",")
        df1["std"] = 0.0
        df1.set_index('SecuCode', inplace=True)
        df2.set_index('SecuCode', inplace=True)
        df = ((df2 - df1) / df2) * 100
        return df
    
    def gen_df(self):
        print(self.getpath_f1)
        print(self.getpath_f2)
        df = self.compare()
        if df.empty == False:
            os.system("pwd") #确认是从yuqin work出发
            os.system("mkdir -p 2020Compare")
            os.chdir("2020Compare")
            os.system("mkdir -p 0120")
            os.chdir("0120")
            os.system("pwd")
            log = open("/home/yuqin/work/2020Compare/0120/%s.csv" %(self.date), "w")
            print('log_open')
            df.to_csv(log, sep = ',')
            log.close() 
            print('log_done')
            os.chdir("../..")
    
class Reports(): # 进一天的全部时间比较文件路径，出一天一个百分比报表
    
    def __init__(self, f, date_i):
        self.f = f
        self.date = date_i
        
    def gen_reports(self):
        # level2summary
        headers = ['AskPriceAtL10Twa', 'AskPriceAtL1Vwa',
               'AskPriceAtL3Twa', 'AskPriceAtL5Twa', 'AskPriceAvgL10Twa', 'AskPriceAvgL10Vwa', 'AskPriceAvgL3Twa', 'AskPriceAvgL3Vwa',
               'AskPriceAvgL5Twa', 'AskPriceAvgL5Vwa', 'AskPriceVwaL10Twa', 'AskPriceVwaL3Twa', 'AskPriceVwaL5Twa', 'AskVolSumL10Twa',
               'AskVolSumL3Twa', 'AskVolSumL5Twa',
              'BidPriceAtL10Twa', 'BidPriceAtL1Vwa', 'BidPriceAtL3Twa', 'BidPriceAtL5Twa', 'BidPriceAvgL10Twa', 'BidPriceAvgL10Vwa',
              'BidPriceAvgL3Twa', 'BidPriceAvgL3Vwa', 'BidPriceAvgL5Twa', 'BidPriceAvgL5Vwa', 'BidPriceVwaL10Twa', 'BidPriceVwaL3Twa',
              'BidPriceVwaL5Twa', 'BidVolSumL10Twa', 'BidVolSumL3Twa', 'BidVolSumL5Twa', 'LastPriceStd', 'PreClosePrice']
#         # orderflow
#         headers = ['BuyAmount', 'BuyExlargeOrders', 'BuyExlargeVolume', 'BuyLargeOrders', 'BuyLargeVolume', 'BuyMedOrders', 
#            'BuyMedVolume', 'BuyOrders', 'BuySmallOrders', 'BuySmallVolume', 'BuyVolume', 'SellAmount', 'SellExlargeOrders', 
#           'SellExlargeVolume', 'SellLargeOrders', 'SellLargeVolume', 'SellMedOrders', 'SellMedVolume', 'SellOrders', 
#           'SellSmallOrders', 'SellSmallVolume', 'SellVolume']
        
        OneDay_logname = "/home/yuqin/work/2020Compare/0120/reports/" + str(self.date) + ".csv"
        OneDay_log = open(OneDay_logname , 'w')
        log = csv.DictWriter(OneDay_log, headers)
        log.writeheader()
        print('log opened')

        df = pd.read_csv(self.f, sep=",")
        df_colname = list(df.columns)
        col = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]  # level2summary
#         col = range(1,len(df_colname)-1) # orderflow
        puredata = df.values 
        print(self.f)
        ratio = []
        for j in range(len(col)):
            ratio.append(0)
            total = 0
            for i in range(puredata.shape[0]):
                total += 1
                if np.isnan(puredata[i][col[j]]) == True:
                    puredata[i][col[j]] = 0  
                if abs(puredata[i][col[j]]) >= 0.1:
                    ratio[j] += 1
            ratio[j] = (ratio[j]/total) *100
            # level2summary
        log.writerow({'AskPriceAtL10Twa':str(ratio[0])+'%', 'AskPriceAtL1Vwa':str(ratio[1])+'%', 
                      'AskPriceAtL3Twa':str(ratio[2])+'%', 'AskPriceAtL5Twa':str(ratio[3])+'%', 'AskPriceAvgL10Twa':str(ratio[4])+'%', 
                      'AskPriceAvgL10Vwa':str(ratio[5])+'%', 'AskPriceAvgL3Twa':str(ratio[6])+'%','AskPriceAvgL3Vwa':str(ratio[7])+'%', 
                      'AskPriceAvgL5Twa':str(ratio[8])+'%', 'AskPriceAvgL5Vwa':str(ratio[9])+'%', 'AskPriceVwaL10Twa':str(ratio[10])+'%', 
                      'AskPriceVwaL3Twa':str(ratio[11])+'%', 'AskPriceVwaL5Twa':str(ratio[12])+'%', 'AskVolSumL10Twa':str(ratio[13])+'%', 
                      'AskVolSumL3Twa':str(ratio[14])+'%', 'AskVolSumL5Twa':str(ratio[15])+'%', 'BidPriceAtL10Twa':str(ratio[16])+'%', 
                      'BidPriceAtL1Vwa':str(ratio[17])+'%', 'BidPriceAtL3Twa':str(ratio[18])+'%', 'BidPriceAtL5Twa':str(ratio[19])+'%', 
                      'BidPriceAvgL10Twa':str(ratio[20])+'%', 'BidPriceAvgL10Vwa':str(ratio[21])+'%', 'BidPriceAvgL3Twa':str(ratio[22])+'%', 
                      'BidPriceAvgL3Vwa':str(ratio[23])+'%', 'BidPriceAvgL5Twa':str(ratio[24])+'%', 'BidPriceAvgL5Vwa':str(ratio[25])+'%', 
                      'BidPriceVwaL10Twa':str(ratio[26])+'%', 'BidPriceVwaL3Twa':str(ratio[27])+'%', 'BidPriceVwaL5Twa':str(ratio[28])+'%', 
                      'BidVolSumL10Twa':str(ratio[29])+'%', 'BidVolSumL3Twa':str(ratio[30])+'%','BidVolSumL5Twa':str(ratio[31])+'%', 
                      'LastPriceStd':str(ratio[32])+'%', 'PreClosePrice':str(ratio[33])+'%'})
#             # orderflow
#         log.writerow({'BuyAmount':str(ratio[0])+'%', 'BuyExlargeOrders':str(ratio[1])+'%',  'BuyExlargeVolume':str(ratio[2])+'%', 
#                   'BuyLargeOrders':str(ratio[3])+'%', 'BuyLargeVolume':str(ratio[4])+'%', 'BuyMedOrders':str(ratio[5])+'%', 'BuyMedVolume':str(ratio[6])+'%', 
#                   'BuyOrders':str(ratio[7])+'%', 'BuySmallOrders':str(ratio[8])+'%', 'BuySmallVolume':str(ratio[9])+'%', 'BuyVolume':str(ratio[10])+'%', 
#                   'SellAmount':str(ratio[11])+'%', 'SellExlargeOrders':str(ratio[12])+'%', 'SellExlargeVolume':str(ratio[13])+'%', 'SellLargeOrders':str(ratio[14])+'%', 
#                  'SellLargeVolume':str(ratio[15])+'%', 'SellMedOrders':str(ratio[16])+'%', 'SellMedVolume':str(ratio[17])+'%', 'SellOrders':str(ratio[18])+'%', 
#                  'SellSmallOrders':str(ratio[19])+'%', 'SellSmallVolume':str(ratio[20])+'%', 'SellVolume':str(ratio[21])+'%'})
        print('log done')


def main():
    
#     for year in range(2012, 2021):
#         datelist = os.listdir("/dat/all_ref/Equity/CN/marketdata/level2summary/5min/data/" + str(year))
#     #往下全部缩进一格, main中以天为单位循环
#     datelist = [20200803,20200806,20200826]
    datelist = os.listdir("/dat/all_ref/Equity/CN/marketdata/level2summary/daily/data/2020" )
    for date_i in datelist:
        # Compare()
        f1 = "/dat/all_till20201230/Equity/CN/marketdata/level2summary/daily/data/2020/" + str(date_i) + "/" + str(date_i) + ".txt"
        f2 = "/dat/all_ref/Equity/CN/marketdata/level2summary/daily/data/2020/" + str(date_i) + "/daily"
        compare_res = Compare(f1, f2, date_i)
        compare_res.gen_df()

        # Reports()
        f = "/home/yuqin/work/2020Compare/0120/" + str(date_i) + ".csv"
        reports = Reports(f, date_i)
        reports.gen_reports()
                
                
if __name__ == '__main__':
    
    main()